In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
import flask
import numpy as np
import cv2
import pytesseract

print("Flask version:", flask.__version__)
print("NumPy version:", np.__version__)
print("OpenCV version:", cv2.__version__)
print("Tesseract version:", pytesseract.get_tesseract_version())

Flask version: 3.1.2
NumPy version: 2.2.6
OpenCV version: 4.12.0
Tesseract version: 5.5.0.20241111


C:\Users\prath\AppData\Local\Temp\ipykernel_9320\4158153490.py:6: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.2. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print("Flask version:", flask.__version__)


In [19]:
import os
import csv
from ocr_utils import process_image_and_get_plate

TEST_FOLDER = 'test_images'
GT_CSV = os.path.join(TEST_FOLDER, 'ground_truth.csv')

rows = []
with open(GT_CSV, newline='') as f:
    reader = csv.DictReader(f)
    for r in reader:
        # Clean up spaces in keys and values
        clean_row = {k.strip(): v.strip() for k, v in r.items()}
        rows.append(clean_row)

for r in rows:
    # Use your actual CSV headers
    filename = r['test_images']  
    gt_plate = r['expected_plate']  

    image_path = os.path.join(TEST_FOLDER, filename)

    result = process_image_and_get_plate(image_path)
    pred_plate = result['plate']
    raw_text = result['raw']
    valid = result['valid']

    print(f"{filename} | GT: {gt_plate} -> Pred: {pred_plate} | Valid: {valid} | Raw: {raw_text}")


sample1.jpeg | GT: RJ14CV0002 -> Pred: RJ12CV0002 | Valid: True | Raw: RJ1ZCV0002
sample2.jpeg | GT: KA20ES2473 -> Pred:  | Valid: False | Raw: 
sample3.jpeg | GT: MH20DV2366 -> Pred: MH200V2366 | Valid: False | Raw: MHZ00V2366
sample4.jpeg | GT: HR06AY1229 -> Pred: HR0GAY1229 | Valid: False | Raw: HROGAY1229
sample5.jpeg | GT: TS09AB1234 -> Pred: T509AB1234 | Valid: False | Raw: TSO9AB1234
sample6.jpeg | GT: TN88F4089 -> Pred: R0 | Valid: False | Raw: RO
sample7.jpeg | GT: WB06F5977 -> Pred:  | Valid: False | Raw: 
sample8.jpeg | GT: TN88F4089 -> Pred:  | Valid: False | Raw: 


In [16]:
def levenshtein(a, b):
    if a == b:
        return 0
    la, lb = len(a), len(b)
    dp = [[0]*(lb+1) for _ in range(la+1)]
    for i in range(la+1): dp[i][0]=i
    for j in range(lb+1): dp[0][j]=j
    for i in range(1, la+1):
        for j in range(1, lb+1):
            cost = 0 if a[i-1]==b[j-1] else 1
            dp[i][j] = min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
    return dp[la][lb]

lev_dist = levenshtein(pred_plate, expected_plate)
max_len = max(len(pred_plate), len(expected_plate))
char_level_acc = 1 - lev_dist/max_len

In [15]:
if pred_plate == expected_plate:
    exact_match = True
else:
    exact_match = False

print(f"Prediction Match: {exact_match}")

Prediction Match: False


In [12]:
expected_plate = r['expected_plate'].strip().upper()

In [13]:
if pred_plate == expected_plate:
    exact_match = True
else:
    exact_match = False

In [4]:
print(rows[0].keys())

dict_keys(['test_images', 'expected_plate'])


In [2]:
import os
import csv

TEST_FOLDER = 'test_images'
GT_CSV = os.path.join(TEST_FOLDER, 'ground_truth.csv')

# Load CSV data
rows = []
with open(GT_CSV, newline='') as f:
    reader = csv.DictReader(f)
    for r in reader:
        r = {k.strip(): v.strip() for k, v in r.items()}
        rows.append(r)

# Now you can print the keys
print(rows[0].keys())

dict_keys(['test_images', 'expected_plate'])
